

**Make sure that you are connected to the kernel associated with our virtual environment . Go to `Kernel` -> `Change kernel` and choose `pipa_venv`.**

In [1]:
import pandas as pd
import requests

# Load the main dataset of ancient Greek texts

In [2]:
# find local version of the input dataset
local_paths = !find ~/Projects -name "LIRE_v1-0.json"
print(local_paths)

['/Users/kasev/Projects/paul/data/large_files/LIRE_v1-0.json']


In [3]:
# LAGT (v1.0.1) dataset directly from Zenodo
try:
    LAGT = pd.read_json(local_paths[0])
except:
    resp = requests.get("https://zenodo.org/record/4971946/files/LAGT_v1-0.json?download=1")
    # save it for next time
    LAGT = pd.DataFrame(resp.json())
    LAGT.to_json("../data/large_files/LIRE_v1-0.json")

In [4]:
# get some overview of the shape of the data
LAGT.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,n_sentences,lemmatized_sentences
0,tlg0001.tlg001.perseus-grc2.xml,Apollonius Rhodius,Argonautica,38822,tlg0001,tlg0001.tlg001,3 B.C.,-2.5,{'-2.5': 1},-2.5,pagan,Epici/-ae,"ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μ...",3252,"[[ἄρχω, Φοῖβος, παλαιγενής, κλέος, φώς, μιμνήσ..."
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Historici/-ae,Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,6068,"[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ..."
2,tlg0004.tlg001.perseus-grc1.xml,Diogenes Laertius,Lives of Eminent Philosophers,110763,tlg0004,tlg0004.tlg001,A.D. 3,2.5,{'2.5': 1},NaN,,Biographi,Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβάρ...,10245,"[[φιλοσοφία, ἔργον, ἔνιοι, φημί, βάρβαρος, ἄρχ..."
3,tlg0005.tlg001.perseus-grc1.xml,Theocritus,Idylls,19200,tlg0005,tlg0005.tlg001,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,"̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα,...",1982,"[[αδύ, ψιθύρισμα, πίτυς, αἰπόλος, τῆνος, πηγή,..."
4,tlg0005.tlg002.perseus-grc1.xml,Theocritus,Epigrams,1734,tlg0005,tlg0005.tlg002,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ἕ...,152,"[[ῥόδον, δροσόεντα, κατάπυκνος, ἐκεῖνος, ἕρπυλ..."


In [5]:
def flat_lemmata(sentences):
    lemmata = [word for sent in sentences for word in sent]
    return lemmata
LAGT["lemmata"] = LAGT["lemmatized_sentences"].apply(flat_lemmata)

LAGT["lemmata_wordcount"] = LAGT["lemmata"].apply(lambda x: len(x))
LAGT["lemmata_wordcount"].sum() # previously we had 13925726, then 13713183, finally 14756899

14383627

# Extract subcorpora



In [6]:
### how many documents we have
len(LAGT)

1457

In [18]:
### let's identify our main author of interest
c_hippocraticum = LAGT[LAGT["author_id"].str.startswith("tlg0627")]
len(c_hippocraticum) ### old value: 53

52

In [16]:
c_platonicum = LAGT[LAGT["author_id"].str.startswith("tlg0059")]
len(c_platonicum)

37

In [7]:
# for comparison, we also generate corpus aristotelicum
c_aristotelicum = LAGT[LAGT["author_id"].str.startswith("tlg0086")]
len(c_aristotelicum) ### originally we had 27

35

In [10]:
# However in the case of Aristotle, we are interested only in a subselection of works associated with his name
# we coded them manually in the "include" column
c_aristotelicum_coded = pd.read_csv("../data/c_aristotelicum_OVERVIEW.csv")

In [11]:
### we can check whether the new and old version have the same order
print(c_aristotelicum_coded["doc_id"].tolist())
print(c_aristotelicum["doc_id"].tolist())

['tlg0086.tlg001', 'tlg0086.tlg002', 'tlg0086.tlg003', 'tlg0086.tlg005', 'tlg0086.tlg006', 'tlg0086.tlg008', 'tlg0086.tlg009', 'tlg0086.tlg010', 'tlg0086.tlg013', 'tlg0086.tlg014', 'tlg0086.tlg015', 'tlg0086.tlg016', 'tlg0086.tlg017', 'tlg0086.tlg018', 'tlg0086.tlg020', 'tlg0086.tlg021', 'tlg0086.tlg022', 'tlg0086.tlg024', 'tlg0086.tlg025', 'tlg0086.tlg026', 'tlg0086.tlg029', 'tlg0086.tlg030', 'tlg0086.tlg031', 'tlg0086.tlg034', 'tlg0086.tlg035', 'tlg0086.tlg037', 'tlg0086.tlg038', 'tlg0086.tlg040', 'tlg0086.tlg041', 'tlg0086.tlg042', 'tlg0086.tlg043', 'tlg0086.tlg044', 'tlg0086.tlg045', 'tlg0086.tlg052', 'tlg0086.tlg054']
['tlg0086.tlg001', 'tlg0086.tlg002', 'tlg0086.tlg003', 'tlg0086.tlg005', 'tlg0086.tlg006', 'tlg0086.tlg008', 'tlg0086.tlg009', 'tlg0086.tlg010', 'tlg0086.tlg013', 'tlg0086.tlg014', 'tlg0086.tlg015', 'tlg0086.tlg016', 'tlg0086.tlg017', 'tlg0086.tlg018', 'tlg0086.tlg020', 'tlg0086.tlg021', 'tlg0086.tlg022', 'tlg0086.tlg024', 'tlg0086.tlg025', 'tlg0086.tlg026', 'tlg0086

In [12]:
# 1) add the "include?" column from the old overview to our new dataset 
c_aristotelicum["include?"] = c_aristotelicum_coded["include?"].tolist()
# 2) use the "include?" column for filtering the new dataset
# 3) drop "include?" column 
c_aristotelicum = c_aristotelicum[c_aristotelicum["include?"]=="y"].drop("include?", axis=1)
c_aristotelicum.head()

/var/folders/zg/zvg9y3rs7j527jxfq9sc2xqc0000gn/T/ipykernel_53043/981519316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_aristotelicum["include?"] = c_aristotelicum_coded["include?"].tolist()


,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,n_sentences,lemmatized_sentences,lemmata,lemmata_wordcount
685,tlg0086.tlg001.1st1K-grc2.xml,Aristotle,Aristotelis Analytica Priora et Posteriora,59614,tlg0086,tlg0086.tlg001,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,Philosophici/-ae,Πρῶτον εἰπεῖν περὶ τί καὶ τίνος ἐστὶν ἡ σκέψις...,3710,"[[πρῶτος, λέγω, εἰμί, σκέψις, ἀπόδειξις, ἐπιστ...","[πρῶτος, λέγω, εἰμί, σκέψις, ἀπόδειξις, ἐπιστή...",24680
686,tlg0086.tlg002.1st1K-grc2.xml,Aristotle,De anima,20912,tlg0086,tlg0086.tlg002,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,Philosophici/-ae,Τῶν καλῶν καὶ τιμίων τὴν εἴδησιν ὑπολαμβάνοντε...,1317,"[[καλός, τίμιος, εἶδος, ὑπολαμβάνω, ἕτερος, ἕτ...","[καλός, τίμιος, εἶδος, ὑπολαμβάνω, ἕτερος, ἕτε...",9395
688,tlg0086.tlg005.1st1K-grc1.xml,Aristotle,De caelo,30794,tlg0086,tlg0086.tlg005,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,Philosophici/-ae,Ἡ περὶ φύσεως ἐπιστήμη σχεδὸν ἡ πλείστη φαίνετ...,2276,"[[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγε...","[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγεθ...",13437
689,tlg0086.tlg006.1st1K-grc1.xml,Aristotle,Categoriae,10316,tlg0086,tlg0086.tlg006,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,Philosophici/-ae,"ὉΜΩΝΥΜΑ λέγεται ὧν ὄνομα μόνον κοινόν, ὁ δὲ κα...",660,"[[ὁμώνυμος, λέγω, ὄνομα, μόνος, κοινός, ὄνομα,...","[ὁμώνυμος, λέγω, ὄνομα, μόνος, κοινός, ὄνομα, ...",4678
690,tlg0086.tlg008.1st1K-grc1.xml,Aristotle,De divinatione per somnum,1194,tlg0086,tlg0086.tlg008,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,Philosophici/-ae,Περὶ δὲ τῆς μαντικῆς τῆς ἐν τοῖς ὕπνοις γινομέ...,68,"[[μαντικός, ὕπνος, γίγνομαι, λέγω, συμβαίνω, ἐ...","[μαντικός, ὕπνος, γίγνομαι, λέγω, συμβαίνω, ἐν...",549


In [13]:
len(c_aristotelicum)

27

# Export the corpora

In [19]:
c_hippocraticum.to_json("../data/c_hippocraticum.json")
c_platonicum.to_json("../data/c_platonicum.json")
c_aristotelicum.to_json("../data/c_aristotelicum.json")